In [48]:
import tvm_users_scatter as tus
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

In [132]:
q_current = '''
select
    ch.category,
    ch.name as channel,
    sum(ss.clip_duration)/60 as tvm,
    count(distinct anon_user_id) users
from test_pluto_clip_start_stop_times ss
join pluto_channels ch
    on ch.id = ss.channel_id
    and ch.direct_only = true
    and ss.request_date >= current_date - 30
    and ss.platform not in ('Web','Unknown','Embed','Desktop')
group by 1,2
'''

q_previous = '''
select
    ch.category,
    ch.name as channel,
    sum(ss.clip_duration)/60 as tvm,
    count(distinct anon_user_id) users
from test_pluto_clip_start_stop_times ss
join pluto_channels ch
    on ch.id = ss.channel_id
    and ch.direct_only = true
    and ss.request_date between current_date - 60 and current_date - 31
    and ss.platform not in ('Web','Unknown','Embed','Desktop')
group by 1,2
'''

In [24]:
df = tus.queryData(query=q_current)
df = tus.cleanDF(df)

## Visualization Guide
- The scatter plot shows USERS on the horizontal axis, and TVMs on the vertical axis.

#### Understanding the Zones:
- Under-Performers (bad):
    - If a channel is in the Under-Performers zone, it has a high # of users, but low TVMs. What this means is we're getting a lot of users to watch the channel (maybe it's positioned well on the dial, got featured, has a broadly appealing channel name etc.), but the content is not sticking with them. In other words, each user isn't staying around to watch very long on that channel, despite the fact that we get a whole ton of users there.
- Over-Achievers (good):
    - If a channel is in the Over-Achievers zone, it has a low # of users, but high TVMs. What this means is we're not getting a lot of users to the channel (maybe it's not positioned well on the dial, didn't get featured, or just a niche type of channel, etc.) but the content is extremely sticky. In other words, each user is staying around to watch the channel for a long time whenever they come to it, despite a smaller portion of users actually land on it.
- Other Interpretations:
    - If a channel is in the top right part of the graph:
        - If it's above the Benchmark Line, this is the BEST CASE scenario.
        - If it's slightly below the Benchmark Line, we can let that slide, because it has done well to get a large number of both users and TVMs.
    - If a channel is in the bottom left part of the graph, it's either a new channel that didn't get enough time to accumulate viewership in this graph's time range, or it attracted neither viewers nor TVMs (WORST CASE scenario).

#### Understanding the Lines:
- Benchmark Line (black):
    - This is the benchmark line. If you had a hypothetical channel and it had the 'average' benchmark performance, it would be somewhere on this line.
- Above-Average Benchmark Line (grey):
    - This is one standard deviation <u>above</u> the average benchmark line. Channels between this line and the Benchmark Line have an above-average performance, but not outstanding enough to call out. Channels beyond this line can be considered "Over-Achievers" because of their superior performance.
- Below-Average Benchmark Line (grey):
    - This is one standard deviation <u>below</u> the average benchmark line. Channels between this line and the Benchmark Line have an above-average performance, but not outstanding enough to call out. Channels beyond this line can be considered "Under-Performers" because of their underwhelming performance.

In [131]:
reload(tus)
p = tus.createViz(df)

Loading BokehJS ...

In [135]:
# df2 = tus.queryData(query=q_previous)
# df2 = tus.cleanDF(df2)

In [136]:
# reload(tus)
# p = tus.createViz(df)